# Make Web Site

In [37]:
import sys, os, io, datetime
import json
import random
import requests
import zipfile
import pandas as pd
from slugify import slugify
from tqdm.notebook import tqdm
from html import escape

In [2]:
import cdli

In [3]:
pd.set_option("display.max_columns", None)

In [4]:
supported_langs = set(["akk"])

In [5]:
wwwroot = os.path.abspath("../dist")
os.makedirs(wwwroot, exist_ok=True)
wwwroot

'/Users/fak/Dropbox/Projects/CuneiformTranslators/dist'

## Download the Catalog

In [ ]:
print("Downloading catalog...")

In [6]:
cat = cdli.get_catalog()

/Users/fak/Dropbox/Projects/CuneiformTranslators/tools/cdli.py:95: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,13,14,16,17,19,22,23,24,25,26,27,28,29,31,32,33,37,39,42,45,46,49,52,53,54,55,56,57,58,59,60,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  cat = pd.read_csv(io.StringIO(cat_csv))


In [7]:
cat

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
0,NaN,NaN,NaN,NaN,21198/zz001q0dtm,"Englund, Robert K.",NaN,CDLI,"31x61x18; Lú A 14-16.30-32.48-50; M XVIII, auf...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2020-03-14,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 065",NaN,NaN,NaN,NaN,"W 06435,a",NaN,auf Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,31,1,0,1,NaN,undetermined,150ppi 20160630,clay,VAT 01533,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 065",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,18,no translation,61,NaN
1,NaN,NaN,NaN,NaN,21198/zz001q0dv4,"Englund, Robert K.",NaN,CDLI,30x48x13; Lú A 13-15.23-25.?; Fundstelle wie W...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 066",NaN,NaN,NaN,NaN,"W 06435,b",NaN,auf der Hügeloberfläche in der Nähe des Südbaues,"M XVIII,?",Lexical,NaN,NaN,30,2,0,2,NaN,undetermined,150ppi 20160630,clay,VAT 15263,NaN,tablet,Uruk III (ca. 3200-3000 BC),NaN,NaN,"CDLI Lexical 000002, ex. 066",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,13,no translation,48,NaN
2,NaN,NaN,NaN,NaN,21198/zz001q0dwn,"Englund, Robert K.",NaN,"Englund, Robert K. & Nissen, Hans J.","42x53x19; Vocabulary 9; Qa XVI,2, unter der Ab...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",NaN,NaN,12/4/2001,NaN,NaN,2020-01-26,NaN,20011204 protocuneiform_catalogue,"ATU 3, pl. 081, W 9123,d",NaN,NaN,NaN,NaN,"W 09123,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,42,3,0,3,NaN,undetermined,150ppi 20160630,clay,VAT 15253,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"ATU 3, pl. 081, W 9123,d",Uruk (mod. Warka),NaN,1993,"ATU 1, 539",NaN,NaN,NaN,NaN,Archaic Vocabulary (witness),Text category: 15-09; Foreign ID: LVO 9,NaN,NaN,19,no translation,53,NaN
3,NaN,NaN,NaN,NaN,21198/zz001q0dx5,"Englund, Robert K.",NaN,CDLI,26x23x23; Lú A 9-10.?.?; Fundstelle wie W 9123...,NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 051",NaN,NaN,NaN,NaN,"W 09169,d",NaN,"unter der Abgleichung der Schicht III, 1,5 m ü...","Qa XVI,2",Lexical,NaN,NaN,26,4,0,4,NaN,undetermined,150ppi 20160630,clay,VAT 15168,NaN,tablet,Uruk IV (ca. 3350-3200 BC),NaN,NaN,"CDLI Lexical 000002, ex. 051",Uruk (mod. Warka),NaN,2015ff.,"Englund, Robert K. & Nissen, Hans J., ATU 3 (1...",NaN,NaN,NaN,NaN,Archaic Lu2 A (witness),NaN,NaN,NaN,23,no translation,23,NaN
4,NaN,NaN,NaN,NaN,21198/zz001q0dzp,"Englund, Robert K.",NaN,CDLI,"29x36x20; Lú A Vorläufer; Qa XVI,2, unter der ...",NaN,NaN,NaN,"Vorderasiatisches Museum, Berlin, Germany",Q000002,NaN,12/4/2001,00.00.00.00,NaN,2018-10-20,00.00.00.00,20011204 protocuneiform_catalogue,"CDLI Lexical 000002, ex. 172",NaN,NaN,NaN,NaN,"W 09206,k",

In [8]:
len(cat)

352771

In [9]:
cat.columns

Index(['accession_no', 'accounting_period', 'acquisition_history',
       'alternative_years', 'ark_number', 'atf_source', 'atf_up', 'author',
       'author_remarks', 'cdli_collation', 'cdli_comments', 'citation',
       'collection', 'composite_id', 'condition_description', 'date_entered',
       'date_of_origin', 'date_remarks', 'date_updated', 'dates_referenced',
       'db_source', 'designation', 'dumb', 'dumb2', 'electronic_publication',
       'elevation', 'excavation_no', 'external_id', 'findspot_remarks',
       'findspot_square', 'genre', 'google_earth_collection',
       'google_earth_provenience', 'height', 'id', 'id_text2', 'id_text',
       'join_information', 'language', 'lineart_up', 'material', 'museum_no',
       'object_preservation', 'object_type', 'period', 'period_remarks',
       'photo_up', 'primary_publication', 'provenience', 'provenience_remarks',
       'publication_date', 'publication_history', 'published_collation',
       'seal_id', 'seal_information', 's

In [10]:
genres = sorted(list(set([str(x).replace("?", "").strip().lower() for x in cat["genre"].unique()])))
genres

['administative',
 'administrative',
 'administrative record',
 'astronomical',
 'astronomical, omen',
 'fake (modern)',
 'historical',
 'legal',
 'letter',
 'lexical',
 'lexical; literary',
 'lexical; literary; mathematical',
 'lexical; mathematical',
 'lexical; school',
 'literary',
 'literary; administrative',
 'literary; letter',
 'literary; lexical',
 'literary; mathematical',
 'mathematical',
 'medical',
 'nan',
 'omen',
 'omens',
 'other (see subgenre)',
 'pottery (seal)',
 'prayer/incantation',
 'private/votive',
 'ritual',
 'royal/monumental',
 'royal/monumental; literary',
 'royal/votive',
 'school',
 'school; literary',
 'scientific',
 'uncertain',
 'votive']

In [11]:
def sanitize_object_type(x):
    return str(x).strip().lower().replace(" (see object remarks)","").replace(" (not impression)", "").replace(" & envelope", "").replace(" in envelope", "").replace("prismatic cylinder", "prism")

object_types = set(sanitize_object_type(x) for x in cat["object_type"].unique())
object_types.remove("nan")
object_types.remove("other")

def get_object_type(x):
    x = sanitize_object_type(x)
    if x in object_types:
        return x
    return "other-object"

print(object_types)
print(get_object_type(" prismatic Cylinder "))

{'brick', 'barrel', 'cylinder', 'cone', 'tablet', 'seal', 'bulla', 'vessel', 'lentil', 'envelope', 'block', 'prism', 'vase', 'docket', 'sealing', 'tag'}
prism


https://cdli.ucla.edu/search/archival_view.php?ObjectID=P256681

In [12]:
cat[cat["id_text"]==256681]

,accession_no,accounting_period,acquisition_history,alternative_years,ark_number,atf_source,atf_up,author,author_remarks,cdli_collation,cdli_comments,citation,collection,composite_id,condition_description,date_entered,date_of_origin,date_remarks,date_updated,dates_referenced,db_source,designation,dumb,dumb2,electronic_publication,elevation,excavation_no,external_id,findspot_remarks,findspot_square,genre,google_earth_collection,google_earth_provenience,height,id,id_text2,id_text,join_information,language,lineart_up,material,museum_no,object_preservation,object_type,period,period_remarks,photo_up,primary_publication,provenience,provenience_remarks,publication_date,publication_history,published_collation,seal_id,seal_information,stratigraphic_level,subgenre,subgenre_remarks,surface_preservation,text_remarks,thickness,translation_source,width,object_remarks
106190,NaN,NaN,NaN,NaN,21198/zz001s1m1w,"de Ridder, Alba",NaN,"Stol, Marten",NaN,NaN,NaN,NaN,University of Pennsylvania Museum of Archaeolo...,NaN,NaN,2/24/2005,NaN,NaN,2021-06-12,NaN,20050224 fitzgerald_upenn,"AbB 11, 029",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Letter,NaN,NaN,?,106192,0,256681,NaN,Akkadian,NaN,clay,UM 29-16-076,NaN,tablet,Old Babylonian (ca. 1900-1600 BC),NaN,600ppi 20160630,"AbB 11, 029",Nippur (mod. Nuffar),NaN,1986,NaN,NaN,NaN,NaN,NaN,NaN,Letter; 7x8x1 line,NaN,NaN,?,no translation,?,NaN


## Get the Transliterations

In [50]:
print("Downloading transliterations...")

In [13]:
transliterations = cdli.get_atf()

100%|██████████████████████████████████████████████████████████████████████████████████| 3541754/3541754 [00:14<00:00, 252318.16it/s]


## Get the Translations

In [16]:
translations_json_path = "../data/ml_translations.json"
translations = json.loads(str(open(translations_json_path, "rb").read(), "utf8"))
translations = translations["akk_to_en"]
len(translations)

13586

## Get a list of publications that we have data for

In [14]:
output_pubs = []

for pub in tqdm(transliterations):
    if pub.language not in supported_langs:
        continue
    pid = pub.id
    if len(pid) < 3 or pid[0] != "P":
        continue
    if pid[-1] == "=":
        pid = pid[:-1]
    pid = int(pid[1:])
    meta = cat[cat["id_text"]==pid]
    if len(meta) != 1:
        continue
    pub.id = pid
    pub.genre = str(meta["genre"].values[0])    
    pub.object_type = str(meta["object_type"].values[0])
    pub.material = str(meta["material"].values[0])
    pub.period = str(meta["period"].values[0])
    pub.provenience = str(meta["provenience"].values[0])
    pub.collection = str(meta["collection"].values[0])
#     language = str(meta["language"].values[0])
#     print(pub.genre)
    output_pubs.append(pub)

len(output_pubs)

  0%|          | 0/134560 [00:00<?, ?it/s]

121111

In [18]:
num_publications = len(cat)
num_transliterations = len(output_pubs)
num_translations = len([x for x in output_pubs if x.has_translations()])
print("    num_publications:", num_publications)
print("num_transliterations:", num_transliterations)
print("    num_translations:", num_translations)

    num_publications: 352771
num_transliterations: 121111
    num_translations: 5196


## Find Publications we have translations for

In [25]:
src_lang = "akk"
tgt_lang = "en"

translated_pubs = []
newly_translated_pubs = []

for pub in tqdm(output_pubs):
    if pub.language != src_lang:
        continue
    has_new_translations = False
    has_ml_translations = True
    for a in pub.text_areas:
        for l in a.lines:
            s = l.text
            if len(s) > 0:
                has_new_translations = has_new_translations or (tgt_lang not in l.languages)
                if s in translations:
                    l.languages["ml_"+tgt_lang] = translations[s]
                else:
                    has_ml_translations = False
    if has_ml_translations:
        translated_pubs.append(pub)
        if has_new_translations:
            newly_translated_pubs.append(pub)
    
            
print(len(translated_pubs), "translated_pubs")
print(len(newly_translated_pubs), "newly_translated_pubs")

  0%|          | 0/121111 [00:00<?, ?it/s]

1246 translated_pubs
752 newly_translated_pubs


In [28]:
newly_translated_pubs[0]

Publication(292940, 'akk', [TextArea('tablet', []), TextArea('obverse', [TextLine('1.', '_1(u) 2(asz) gur sze_ hu-bu-ta-tum', {'ml_en': '12 gur barley of the new-year'}), TextLine('2.', 'u2-sze-ti-iq-ma', {'ml_en': 'he caused to be evicted, and'}), TextLine('3.', '_1(asz) gur 1(barig) 4(ban2) masz2_ u2-s,a-ab', {'ml_en': '1 gur 1 barig 4 ban2 of goat hair will be removed'}), TextLine('4.', '_ki_ {d}utu', {'ml_en': 'with Shamash'}), TextLine('5.', 'u3 gi-da-nu-um', {'ml_en': 'and the scepter'}), TextLine('6.', '{disz}{d}be-el-gesztu-a-bu-szu', {'ml_en': 'Bl-geshtu-abusshu'}), TextLine('7.', '_dumu_ i3-li2-ma-ra-x', {'ml_en': 'son of Ili-mr...'}), TextLine('8.', '_szu ba#-an-ti#_', {'ml_en': 'received'}), TextLine('9.', 'a-na _masz-gan2_-nim#', {'ml_en': 'for the harvest'})]), TextArea('reverse', [TextLine('1.', 'sze-am _i3-ag2-e_', {'ml_en': 'the barley he shall weigh out'}), TextLine('2.', '_igi_ hu-za-lum _dumu_ be-el-szu-nu', {'ml_en': 'before Huzlum, son of their owner'}), TextLine(

## Output the HTML

In [40]:
def get_file_path(site_path):
    return f"{wwwroot}{site_path}"

def get_page_file_path(site_path):
    return f"{get_file_path(site_path)}.html"
    

In [43]:
def header(title, f):
    f.write(f"<html>\n<head>\n<title>{escape(title)}</title>\n</head>\n<body>\n<h1>{escape(title)}</h1>\n")
    
def footer(f):
    f.write(f"</body>\n</html>")

In [49]:
def output_pub(p, f):
    f.write(f"<h1>P{p.id:06}</h1>\n")
    for a in p.text_areas:
        if len(a.lines) == 0:
            continue
        f.write(f"<h2>{escape(a.name)}</h2>\n")
        f.write("<table>")
        for l in a.lines:
            f.write("<tr>")
            f.write(f"<td>{escape(l.text)}</td>\n")
            if "ml_en" in l.languages:
                f.write(f"<td>{escape(l.languages['ml_en'])}</td>\n")
            else:
                f.write(f"<td></td>\n")
            f.write("</tr>")
        f.write("</table>")

print("Writing /new")
with open(get_page_file_path("/new"), "wt") as f:
    header("New Translations", f)
    for p in tqdm(newly_translated_pubs):
        f.write(f"<section>\n")
        output_pub(p, f)        
        f.write(f"</section>\n")
    footer(f)
    
print("Writing /")
with open(get_page_file_path("/index"), "wt") as f:
    header("ML Cuneiform Translations", f)
    f.write(f"<a href=\"/new\">New Translations</a>\n")
    footer(f)

  0%|          | 0/752 [00:00<?, ?it/s]

In [36]:
for pub in tqdm(translated_pubs):
    page_site_path = f"/p{pub.id:06}"
    page_file_path = get_page_file_path(page_site_path)
#     print(page_file_path)



  0%|          | 0/1246 [00:00<?, ?it/s]

/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p292940.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p005984.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p491234.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p481778.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p509349.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p508116.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p508121.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p508161.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p283609.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p372250.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p373021.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p399874.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p373020.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p514350.html
/Users/fak/Dropbox/Projects/CuneiformTranslators/dist/p020464.